# Banking and Unemployment
---

In [44]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import time
from census import Census
from us import states
import gmaps

# census key
from config import (census_key, gkey)

# Census API Key
c = Census(census_key, year=2013)

## Data Retrieval

In [34]:
# Run Census Search to retrieve data on all zip codes (2013 ACS5 Census)
# See: https://github.com/CommerceDataService/census-wrapper for library documentation
# See: https://gist.github.com/afhaque/60558290d6efd892351c4b64e5c01e9b for labels
census_data = c.acs5.get(("B01003_001E", "B23025_005E"), {'for': 'zip code tabulation area:*'})

# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

# Column Reordering
census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                      "B23025_005E": "Unemployment Count",
                                      "zip code tabulation area": "Zipcode"})

# Add in unemployment Rate (Unemployment Count / Population)
census_pd["Unemployment Rate"] = census_pd["Unemployment Count"].astype(int) / census_pd["Population"].astype(int) * 100

census_pd[["Zipcode", "Population", "Unemployment Rate"]]
# Final DataFrame
census_pd.head()

# Visualize

,Population,Unemployment Count,Zipcode,Unemployment Rate
0,18450.0,1464.0,00601,7.934959
1,41302.0,4767.0,00602,11.541814
2,53683.0,4341.0,00603,8.086359
3,6591.0,134.0,00606,2.033075
4,28963.0,1134.0,00610,3.915340


In [41]:
census_pd["Zipcode"] = census_pd["Zipcode"].astype(int)

## Combine Data

In [38]:
# Import the original data we analyzed earlier. Use dtype="object" to match other
census_data_original = pd.read_csv("../Resources/zip_bank_data.csv")
census_data_original["Zipcode"] = census_data_original["Zipcode"].astype(int)

# Visualize

In [39]:
census_data_original.head()

,Zipcode,Address,Population,Median Age,Household Income,Per Capita Income,Poverty Rate,Lat,Lng,Bank Count
0,49058,"Hastings, MI 49058, USA",19031,40.8,46777.0,22137,13.183753,42.630692,-85.292938,9
1,71405,"Ball, LA 71405, USA",5687,35.8,55242.0,23941,15.737647,31.406180,-92.396174,12
2,20634,"Great Mills, MD 20634, USA",6562,31.9,79944.0,35961,9.295946,38.220161,-76.496792,9
3,38487,"Williamsport, TN 38487, USA",762,41.6,38125.0,18884,24.409449,35.731037,-87.241930,0
4,58466,"Marion, ND 58466, USA",509,44.5,69844.0,36981,9.430255,46.559422,-98.348154,1


In [42]:
# Merge the two data sets along zip code
census_complete = pd.merge(census_data_original, census_pd, how="left", on=["Zipcode", "Zipcode"])

# Save the revised Data Frame as a csv
census_complete.to_csv("../Resources/bank_data_with_unemployment.csv", encoding="utf-8", index=False)

# Visualize
census_complete.head()

,Zipcode,Address,Population_x,Median Age,Household Income,Per Capita Income,Poverty Rate,Lat,Lng,Bank Count,Population_y,Unemployment Count,Unemployment Rate
0,49058,"Hastings, MI 49058, USA",19031,40.8,46777.0,22137,13.183753,42.630692,-85.292938,9,19031.0,1011.0,5.312385
1,71405,"Ball, LA 71405, USA",5687,35.8,55242.0,23941,15.737647,31.406180,-92.396174,12,5687.0,260.0,4.571830
2,20634,"Great Mills, MD 20634, USA",6562,31.9,79944.0,35961,9.295946,38.220161,-76.496792,9,6562.0,266.0,4.053642
3,38487,"Williamsport, TN 38487, USA",762,41.6,38125.0,18884,24.409449,35.731037,-87.241930,0,762.0,78.0,10.236220
4,58466,"Marion, ND 58466, USA",509,44.5,69844.0,36981,9.430255,46.559422,-98.348154,1,509.0,6.0,1.178782


## Heatmap of poverty rate

In [45]:
# Configure gmaps with API key
gmaps.configure(api_key = gkey)

In [46]:
# Store 'Lat' and 'Lng' into  locations 
locations = census_complete[["Lat", "Lng"]].astype(float)

# Convert Poverty Rate to float and store
# HINT: be sure to handle NaN values
poverty_rate = census_complete["Poverty Rate"].astype(float)

In [47]:
# Create a poverty Heatmap layer
fig = gmaps.figure()
heatmap_layer = gmaps.heatmap_layer(locations, weights=poverty_rate, dissipating=False, max_intensity=10, point_radius=1)

fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [48]:
# Convert bank rate to list
bank_rate = census_complete["Bank Count"].tolist()


In [49]:
# Create bank symbol layer
bank_layer = gmaps.symbol_layer(
    locations,
    fill_color="rgba(0, 150, 0, 0.4)",
    stroke_color="rgba(0, 150, 0, 0.4)",
    scale=2,
    info_box_content=[f"Bank Amount: {bank}" for bank in bank_rate]
)

fig = gmaps.figure()
fig.add_layer(bank_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [50]:
# Create a combined map
fig = gmaps.figure()
fig.add_layer(heatmap_layer)
fig.add_layer(bank_layer)
fig

Figure(layout=FigureLayout(height='420px'))